### What is BERT

Bidirectional Encoder Representation from Transformers

- Encoder Representations: language modeling system, pre-trained with unalbeled data. Then fine-tunning
- from Transformer: based on powerful NLP algorithm. Defines the architecture of BERT.
- Bidirectional: uses with left and right context when dealing with a word. Defines the training process.

- ELMo = bidirectional with LSTM
- OpenAi GPT = Transformers from left to right only
- BERT both (transformers + bidirectional)

link https://medium.com/@wwydmanski/whats-the-difference-between-self-attention-and-attention-in-transformer-architecture-3780404382f3

Self-attention and attention are both mechanisms that allow transformer models to attend to different parts of the input or output sequences when making predictions. These mechanisms are crucial for the performance of transformer models in tasks such as language translation, text summarization, and sentiment analysis, where the model needs to understand the relationships between different words or phrases in the input and output sequences.

Attention refers to the ability of a transformer model to attend to different parts of another sequence when making predictions. This is often used in encoder-decoder architectures, where the encoder vectorizes the input sequence, and the decoder attends to the encoded representation of the whole input when making predictions.

